In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pylab as plt

%matplotlib inline

/Users/zytec/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 数据预处理

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
print('共有{}条数据.'.format(len(df)))

共有891条数据.


In [4]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


#### Pclass 代表社会经济状况，数字越小状况越好 1 = upper, 2 = mid, 3 = lower

In [5]:
df.Pclass.value_counts(dropna=False)

3    491
1    216
2    184
Name: Pclass, dtype: int64

#### Sex male时为1， female时为0

In [6]:
df.Sex.value_counts(dropna=False)

male      577
female    314
Name: Sex, dtype: int64

In [7]:
df.Sex.replace(['male', 'female'], [1, 0], inplace=True)

#### Age 有缺失值，简单用平均值填充

In [8]:
df.Age.isna().sum()

177

In [9]:
df.Age.fillna(df.Age.mean(), inplace=True)

#### Sibsp	在船上的旁系亲属和配偶的数量

In [10]:
df.SibSp.isna().sum()

0

#### Parch	在船上的父母子女的数量

In [11]:
df.Parch.isna().sum()

0

#### Fare	船费

In [12]:
df.Fare.isna().sum()

0

#### Embarked	登船港口 C = Cherbourg, Q = Queenstown, S = Southampton

In [13]:
df.Embarked.value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

In [14]:
df.Embarked.fillna('S', inplace=True)

In [15]:
df.Embarked.replace(['C','Q','S'], [0, 1, 2], inplace=True)

#### 最后的数据

In [16]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,2
5,6,0,3,"Moran, Mr. James",1,29.699118,0,0,330877,8.4583,NaN,1
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,17463,51.8625,E46,2
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,349909,21.0750,NaN,2
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,347742,11.1333,NaN,2
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,237736,30.0708,NaN,0


## 划分训练集和验证集

In [17]:
total_X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].values
total_Y = df['Survived']

#### 随机化

In [18]:
random_indexes = np.random.permutation(len(total_X))

In [19]:
total_X = total_X[random_indexes]
total_Y = total_Y[random_indexes].reshape(-1, 1)

/Users/zytec/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


#### %70训练集, %30验证集

In [20]:
devide_num = int(len(total_X) * 0.3)
dev_X = total_X[:devide_num]
dev_Y = total_Y[:devide_num]
train_X = total_X[devide_num:]
train_Y = total_Y[devide_num:]

print('train_X shape:', train_X.shape)
print('dev_X shape:', dev_X.shape)

train_X shape: (624, 7)
dev_X shape: (267, 7)


## 构建神经网络

In [166]:
def net(lambd, layers_deep, layers_width):
    with tf.name_scope('placeholder'):
        input = tf.placeholder(dtype=tf.float32, shape=(None, train_X.shape[-1]), name='input')
        label = tf.placeholder(dtype=tf.int32, shape=(None, 1), name='label')
    
    layer_input = hidden_output = input
    
    with tf.name_scope('hidden_layers'):
        for layer_num in range(1, layers_deep):
            hidden_output = tf.layers.dense(layer_input,
                                           layers_width,
                                           activation=tf.nn.relu, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(lambd, dtype=tf.float32)), 
                                           name='layer_' + str(layer_num))
                
            layer_input = hidden_output
            
    with tf.name_scope('output'):
        logits = tf.layers.dense(hidden_output,
                                 1,
                                 activation=None, 
                                 kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(lambd, dtype=tf.float32)), 
                                 name='output_layer')
        output = tf.nn.sigmoid(logits)
        
    return input, label, output, logits

In [167]:
def accuracy(output, labels, threshold=0.5):
    predicts = tf.cast(output >= threshold, tf.int32)
    is_rights = tf.equal(predicts, labels)
    accuracy = tf.reduce_mean(tf.cast(is_rights, tf.float32))
    
    return accuracy

In [168]:
def cost(logits, labels):
    labels = tf.cast(labels, tf.float32)
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))

In [205]:
def train_and_evaluate(train_X, train_Y, dev_X, dev_Y, lr, epochs, lambd, 
                       layers_deep, layers_width, accuracy_threshold=0.5, print_epochs=100):
    input_pl, label_pl, output_op, logits_op = net(lambd, layers_deep, layers_width)
    cost_op = cost(logits_op, label_pl)
    accuracy_op = accuracy(output_op, label_pl, accuracy_threshold)
    optimizer_op = tf.train.AdamOptimizer(lr).minimize(cost_op)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    train_accuracys = []
    dev_accuracys = []
    for epoch in range(1, epochs + 1):
        _, train_cost, train_accuracy = sess.run([optimizer_op, cost_op, accuracy_op], 
                                        feed_dict={input_pl: train_X, label_pl: train_Y})
        dev_cost, dev_accuracy = sess.run([cost_op, accuracy_op], 
                                 feed_dict={input_pl: dev_X, label_pl: dev_Y})
        
        if epoch % print_epochs == 0:
            print('epoch:', epoch)
            print('train_cost:{0:.4f}--dev_cost:{1:.4f}--train_accuracy:{2:.4f}--dev_accuracy:{3:.4f}'.format( 
              train_cost, dev_cost, train_accuracy, dev_accuracy))
            print('')
        
            train_accuracys.append(train_accuracy)
            dev_accuracys.append(dev_accuracy)
        
    plt.plot(train_accuracys, label='train_accuracy')
    plt.plot(dev_accuracys, label='dev_accuracy')
    plt.legend()
    plt.xlabel('epochs/{}'.format(print_epochs))
    plt.ylabel('accuracy')

#### 超参数

In [206]:
lr = 0.01
epochs = 3000
labmd = 0.65
layers_deep = 4
layers_width = 14
threshold = 0.5

In [ ]:
tf.reset_default_graph()
train_and_evaluate(train_X, train_Y, dev_X, dev_Y, lr, epochs, labmd, layers_deep, layers_width, threshold)

epoch: 100
train_cost:0.5392--dev_cost:0.5713--train_accuracy:0.7404--dev_accuracy:0.7041

epoch: 200
train_cost:0.4382--dev_cost:0.4797--train_accuracy:0.8189--dev_accuracy:0.8127

epoch: 300
train_cost:0.3780--dev_cost:0.4694--train_accuracy:0.8429--dev_accuracy:0.8015

epoch: 400
train_cost:0.3825--dev_cost:0.4577--train_accuracy:0.8413--dev_accuracy:0.7865

epoch: 500
train_cost:0.3639--dev_cost:0.4912--train_accuracy:0.8526--dev_accuracy:0.7903

epoch: 600
train_cost:0.3502--dev_cost:0.5211--train_accuracy:0.8462--dev_accuracy:0.7603

epoch: 700
train_cost:0.3494--dev_cost:0.5415--train_accuracy:0.8462--dev_accuracy:0.7528

epoch: 800
train_cost:0.3331--dev_cost:0.5694--train_accuracy:0.8654--dev_accuracy:0.7566

epoch: 900
train_cost:0.3237--dev_cost:0.5927--train_accuracy:0.8670--dev_accuracy:0.7715

epoch: 1000
train_cost:0.3073--dev_cost:0.6285--train_accuracy:0.8750--dev_accuracy:0.7678

epoch: 1100
train_cost:0.3513--dev_cost:0.6585--train_accuracy:0.8558--dev_accuracy:0.782

In [201]:
def predict(X, layers_deep, layers_width):
    pass